In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## 1. Fetch data from  website Weather.com and retrieve 10-day weather forecasts for Newark, DE

In [50]:
url = 'https://weather.com/weather/tenday/l/Newark+DE' 
#url = 'https://weather.com/weather/tenday/l/bfe0ee5ea4a1932d9da0e95a73bdd3545f0bda20b3775ab8944f8bb962264863'
resp = requests.get(url)
print(resp.status_code)


200


In [18]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [46]:
import time
import random

In [48]:
weather_data = []

# Locate all the relevant <details> tags
forecast_details = soup.find_all('details', class_='DaypartDetails--DayPartDetail--2XOOV')

for detail in forecast_details:
    # Each <details> tag includes a summary and additional information
    summary = detail.find('summary')
    date = summary.find('h2', class_='DetailsSummary--daypartName--kbngc').text
    condition = summary.find('span', class_='DetailsSummary--extendedData--307Ax').text
    temp_high = summary.find('span', class_='DetailsSummary--highTempValue--3PjlX').text
    temp_low = summary.find('span', class_='DetailsSummary--lowTempValue--2tesQ').text


   
    weather_data.append({
        'date': date,
        'condition': condition,
        'high_temperature': temp_high,
        'low_temperature': temp_low
    })


In [49]:
df = pd.DataFrame(weather_data)
df

,date,condition,high_temperature,low_temperature
0,Tonight,Partly Cloudy,--,66°
1,Thu 20,Partly Cloudy,90°,68°
2,Fri 21,Sunny,93°,72°
3,Sat 22,Partly Cloudy,95°,76°
4,Sun 23,PM Thunderstorms,95°,76°
5,Mon 24,Scattered Thunderstorms,90°,66°
6,Tue 25,Mostly Sunny,87°,70°
7,Wed 26,Partly Cloudy,93°,71°
8,Thu 27,Partly Cloudy,88°,67°
9,Fri 28,Mostly Sunny,89°,67°


## 2. Get the job search results from Linkedin


In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service

In [79]:
chromedriver_path = "/Users/jinbiao.ji/Downloads/chromedriver-mac-x64/chromedriver"  # Change this to your actual path

# Set up the ChromeDriver
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)

url = 'https://www.linkedin.com/jobs/search/?currentJobId=3926124474&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true'
driver.get(url)

time.sleep(5)

In [80]:
# Locate all job cards within the results list
jobs = driver.find_elements(By.CSS_SELECTOR, '.jobs-search__results-list li')

# List to store job information
job_listings = []

# Loop through found job elements
for job in jobs:
    try:
        job_title = job.find_element(By.CSS_SELECTOR, 'a[data-tracking-control-name="public_jobs_jserp-result_search-card"] span').text.strip()
        company = job.find_element(By.CSS_SELECTOR, 'a[data-tracking-control-name="public_jobs_jserp-result_job-search-card-subtitle"]').text.strip()
        location = job.find_element(By.CLASS_NAME, 'job-search-card__location').text.strip()
        
        # Extract the hiring status, if available
        status_elements = job.find_elements(By.CLASS_NAME, 'job-posting-benefits__text')
        status = status_elements[0].text.strip() if status_elements else 'Not specified'

        # Extract the job link (address)
        job_link = job.find_element(By.CSS_SELECTOR, 'a[data-tracking-control-name="public_jobs_jserp-result_search-card"]').get_attribute('href')

        # Using XPath to locate the <time> element and retrieve its 'datetime' attribute and text content
        time_element = job.find_element(By.XPATH, './/time[@class="job-search-card__listdate"]')
        date_posted = time_element.get_attribute('datetime')  # Retrieves the 'datetime' attribute
        date_text = time_element.text.strip()  # Retrieves visible date text like '5 days ago'

        job_listings.append({
            'Job Title': job_title,
            'Company': company,
            'Location': location,
            'Status': status,
            'Job Link': job_link,
            'Date Posted': date_posted,
            'Date Text': date_text  
        })
    except Exception as e:
        print(f"Error processing a job listing: {e}")

# Close the WebDriver
driver.quit()

Error processing a job listing: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//time[@class="job-search-card__listdate"]"}
  (Session info: chrome=126.0.6478.62); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010bc56108 chromedriver + 5169416
1   chromedriver                        0x000000010bc4dfda chromedriver + 5136346
2   chromedriver                        0x000000010b7ca36c chromedriver + 402284
3   chromedriver                        0x000000010b817740 chromedriver + 718656
4   chromedriver                        0x000000010b817a01 chromedriver + 719361
5   chromedriver                        0x000000010b80bff6 chromedriver + 671734
6   chromedriver                        0x000000010b83aadd chromedriver + 862941
7   chromedriver                        0x000000010b80bed8 chromedriver + 

In [88]:
df_jobs = pd.DataFrame(job_listings)
df_jobs.head()

,Job Title,Company,Location,Status,Job Link,Date Posted,Date Text
0,Data Scientist,Rue Gilt Groupe,"Miami, FL",Actively Hiring,https://www.linkedin.com/jobs/view/data-scient...,2024-06-18,1 day ago
1,data scientist - supply chain,Starbucks,"Seattle, WA",Actively Hiring,https://www.linkedin.com/jobs/view/data-scient...,2024-06-14,5 days ago
2,Data Scientist,Constellation,"New York, NY",Actively Hiring,https://www.linkedin.com/jobs/view/data-scient...,2024-06-04,2 weeks ago
3,"Data Scientist, Product Analytics",Etsy,"Brooklyn, NY",Not specified,https://www.linkedin.com/jobs/view/data-scient...,2024-06-13,6 days ago
4,"Data Scientist, Marketing Analytics",Etsy,"Brooklyn, NY",Not specified,https://www.linkedin.com/jobs/view/data-scient...,2024-06-14,5 days ago


In [82]:
df_jobs.to_csv('LinkedIn_Job_Listings.csv')

In [86]:
df_jobs.shape

(45, 7)

### Scroll the page to get more jobs

In [83]:

chromedriver_path = "/Users/jinbiao.ji/Downloads/chromedriver-mac-x64/chromedriver"  # Change this to your actual path

# Set up the ChromeDriver
service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service)

url = 'https://www.linkedin.com/jobs/search/?currentJobId=3926124474&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true'
driver.get(url)


# Allow some time for the initial page to load
time.sleep(5)

# Function to scrape job data
def scrape_jobs():
    job_listings = []
    jobs = driver.find_elements(By.CSS_SELECTOR, '.jobs-search__results-list li')
    for job in jobs:
        try:
            job_title = job.find_element(By.CSS_SELECTOR, 'a[data-tracking-control-name="public_jobs_jserp-result_search-card"] span').text.strip()
            company = job.find_element(By.CSS_SELECTOR, 'a[data-tracking-control-name="public_jobs_jserp-result_job-search-card-subtitle"]').text.strip()
            location = job.find_element(By.CLASS_NAME, 'job-search-card__location').text.strip()
            status_elements = job.find_elements(By.CLASS_NAME, 'job-posting-benefits__text')
            status = status_elements[0].text.strip() if status_elements else 'Not specified'

            # Using XPath to locate the <time> element and retrieve its 'datetime' attribute and text content
            time_element = job.find_element(By.XPATH, './/time[@class="job-search-card__listdate"]')
            date_posted = time_element.get_attribute('datetime')  # Retrieves the 'datetime' attribute
            date_text = time_element.text.strip()  # Retrieves visible date text like '5 days ago'

            job_listings.append({
                'Job Title': job_title,
                'Company': company,
                'Location': location,
                'Status': status,
                'Date Posted': date_posted,
                'Date Text': date_text,
                'Job Link': job_link,
            })
        except Exception as e:
            print(f"Error processing a job listing: {e}")
    return job_listings

# Scrolling to load all jobs
scroll_pause_time = 2  # You can adjust this depending on your internet speed and response time of the website
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Now that all jobs are likely loaded, perform the scraping
all_job_listings = scrape_jobs()



# Close the WebDriver
driver.quit()

df_all_jobs = pd.DataFrame(all_job_listings)
df_all_jobs.to_csv('LinkedIn_All_Job_Listings.csv')

Error processing a job listing: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//time[@class="job-search-card__listdate"]"}
  (Session info: chrome=126.0.6478.62); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000108ad7108 chromedriver + 5169416
1   chromedriver                        0x0000000108acefda chromedriver + 5136346
2   chromedriver                        0x000000010864b36c chromedriver + 402284
3   chromedriver                        0x0000000108698740 chromedriver + 718656
4   chromedriver                        0x0000000108698a01 chromedriver + 719361
5   chromedriver                        0x000000010868cff6 chromedriver + 671734
6   chromedriver                        0x00000001086bbadd chromedriver + 862941
7   chromedriver                        0x000000010868ced8 chromedriver + 

In [85]:
df_all_jobs.shape

(93, 7)

In [84]:
df_all_jobs.head()

,Job Title,Company,Location,Status,Date Posted,Date Text,Job Link
0,data scientist - supply chain,Starbucks,"Seattle, WA",Actively Hiring,2024-06-14,5 days ago,https://www.linkedin.com/jobs/view/data-scient...
1,Data Scientist,Rue Gilt Groupe,"Miami, FL",Actively Hiring,2024-06-18,1 day ago,https://www.linkedin.com/jobs/view/data-scient...
2,"Data Scientist, Product Analytics",Etsy,"Brooklyn, NY",Not specified,2024-06-13,6 days ago,https://www.linkedin.com/jobs/view/data-scient...
3,"Data Scientist, Marketing Analytics",Etsy,"Brooklyn, NY",Not specified,2024-06-14,5 days ago,https://www.linkedin.com/jobs/view/data-scient...
4,Data Scientist,Amdocs,"Seattle, WA",Actively Hiring,2024-06-03,2 weeks ago,https://www.linkedin.com/jobs/view/data-scient...
